<a href="https://colab.research.google.com/github/Mohamm76/Identifying_facial_wrinkles/blob/main/FER_2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# تثبيت الـ kaggle (لو لم يكن مثبتاً) وتأكد من وجود صلاحيات
!pip install -q kaggle

from google.colab import files
uploaded = files.upload()

In [ ]:
!mv "/content/kaggle (1).json" /content/kaggle.json


In [ ]:
from pathlib import Path
import os

kaggle_json_path = Path('/content/kaggle.json')

if kaggle_json_path.exists():
    os.makedirs('/root/.kaggle', exist_ok=True)
    !cp /content/kaggle.json /root/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json
    print('✅ kaggle.json moved to /root/.kaggle and permissions set.')
else:
    print('❌ لم أجد kaggle.json في /content — ارفع ملف kaggle.json ثم أعد تشغيل هذه الخلية.')



In [ ]:
# تنزيل مجموعة بيانات FER-2013 من Kaggle

!kaggle datasets download -d msambare/fer2013 -q --unzip

ما فائدة المولد (ImageDataGenerator)؟

الفكرة الأساسية 👇
بدلًا من تحميل كل الصور إلى الذاكرة دفعة واحدة (وهو مستحيل أحيانًا عندما تكون آلاف الصور)، يقوم المولد بتحميل دفعات صغيرة (batches) من الصور أثناء التدريب فقط.

🔹 أي أنه:

يقرأ الصور من المجلد مباشرة.

يطبّق عليها التحسينات المطلوبة (مثل التدوير، القص، التطبيع).

يرسلها إلى النموذج على شكل دفعات (batch) بشكل مستمر أثناء التدريب.

وهذا ما يُسمّى بـ التغذية الكسولة (lazy loading) أو التغذية أثناء التدريب.

In [ ]:
import os
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


#إنشاء مولد (generator) يقوم بقراءة الصور من المجلدات مع إمكانية تحسينها أو تعديلها (augmentation) أثناء التدريب.
IMG_SIZE = 48 #حجم الصورة المطلوب عند إدخالها للنموذج (كل صورة سيتم تحويلها إلى 48×48 بكسل).
BATCH_SIZE = 32 #عدد الصور التي ستُعالج دفعة واحدة في كل خطوة تدريب (batch size = 32).

# إنشاء مولد للصور مع التوسيع augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, #تطبيع الصور بتحويل القيم من [0,255] إلى [0,1] لتسريع التعلم.
    rotation_range=10, #تدوير الصورة عشوائيًا بزاوية تصل إلى ±10 درجات.
    width_shift_range=0.1, #تحريك الصورة أفقيًا بنسبة 10% من عرضها.
    height_shift_range=0.1,#تحريك الصورة عموديًا بنسبة 10%.
    horizontal_flip=True #قلب الصورة أفقيًا عشوائيًا.

  #الهدف: جعل النموذج يرى تنوعًا أكبر في نفس البيانات (زيادة القدرة على التعميم).
)


'''
🔹 هنا مولد بيانات للاختبار فقط،
لكن بدون أي تعديل (augmentation) لأننا نريد تقييم النموذج على الصور الأصلية فقط.
نكتفي بتطبيع القيم [0,1].
'''
test_datagen = ImageDataGenerator(rescale=1./255)


#هذا السطر ينشئ مولد فعلي للصور التدريبية يقرأ الصور من المجلد training/.
# تحميل البيانات من المجلدات
train_generator = train_datagen.flow_from_directory(
    '/content/train',          # مسار مجلد التدريب
    target_size=(IMG_SIZE, IMG_SIZE), #يغير حجم كل صورة إلى 48×48 بكسل.
    color_mode='grayscale',  # لأن صور FER-2013 أبيض وأسود
    batch_size=BATCH_SIZE, #يعيد 32 صورة في كل دفعة تدريب.
    class_mode='categorical'
    #يعني أن التصنيفات ستكون one-hot encoded (مثل [0,0,1,0,0,0,0] لتصنيف الوجه).
)

#the same train data
test_generator = test_datagen.flow_from_directory(
    '/content/test',             # مسار مجلد الاختبار
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False        # للحفاظ على ترتيب الصور في التقييم

 )

  #shuffle=False: يعني لا نخلط ترتيب الصور
 # (حتى نحافظ على ترتيبها الأصلي عند حساب النتائج أو المقارنة مع التسميات الحقيقية).


EPOCHS = 25

#هنا نحصل على عدد الفئات (classes) تلقائيًا من المولد train_generator.
#train_generator.class_indices → قاموس يربط اسم كل فئة برقم.
#len(...) → عدد الفئات الإجمالي.
#نطبعه لتأكيد عدد أصناف الوجه الموجودة في  dataset
num_classes = len(train_generator.class_indices)
print("عدد الفئات:", num_classes)

# ===== بناء نموذج CNN بسيط =====
input_shape = (IMG_SIZE, IMG_SIZE, 1)#تحديد شكل الصورة الداخلة للنموذج (العرض  الإرتفاع قنات اللون )

model = models.Sequential([
    layers.Input(shape=input_shape),#يحدد شكل المدخلات.

    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    #padding='same' → يضمن بقاء أبعاد الصورة بعد الالتفاف نفسها.
    layers.BatchNormalization(),
    #يحسن سرعة التدريب ويقلل الـ overfitting عن طريق توحيد توزيع القيم داخل كل batch.
    layers.MaxPooling2D((2,2)),
    #تقليل أبعاد الصورة (downsampling) إلى النصف.
    #يحافظ على أهم الميزات المميزة.

    #نفس الفكرة السابقة لكن عدد الفلاتر زاد إلى 64 → تعلم ميزات أكثر تعقيدًا.
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    #Dropout(0.25) → إيقاف 25% من الخلايا عشوائيًا أثناء التدريب لتقليل الـ overfitting.
    layers.Dropout(0.25),

    #عدد الفلاتر أصبح 128 → تمثيل ميزات أكثر تعقيدًا.
    #نفس خطوات Normalization, Pooling, Dropout.
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    #Flatten() → تحويل المخرجات ثنائية الأبعاد من Conv إلى متجه 1D ليتم إدخالها للـ Dense.
    layers.Flatten(),
    #Dense(128, activation='relu') → طبقة fully connected بها 128 خلية لتجميع الميزات.
    layers.Dense(128, activation='relu'),
    #BatchNormalization() + Dropout(0.5) → توحيد القيم وتقليل overfitting.
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    #Dense(num_classes, activation='softmax') → طبقة الإخراج.
    layers.Dense(num_classes, activation='softmax')
])
#تجميع النموذج
#optimizer='adam' → خوارزمية تحسين تلقائية فعالة.
#loss='categorical_crossentropy' → مناسبة لتصنيف متعدد الفئات.
#metrics=['accuracy'] → نراقب دقة النموذج أثناء التدريب.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
# ===== تدريب النموذج =====
callbacks = [
    # تقليل معدل التعلم إذا توقفت الخسارة عن التحسن
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    # إيقاف مبكر عند عدم التحسن
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)
]

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:
# ===== رسم الخسارة والدقة =====
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
# ===== تقييم النموذج =====
test_generator.reset()
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys())))

In [ ]:
# ===== حفظ النموذج =====
model.save('fer2013_cnn_model.h5')
print('✅ النموذج تم حفظه باسم fer2013_cnn_model.h5')

In [1]:
# 1. تثبيت المكتبة أولاً (إذا لم تكن قد ثبتها في هذه الجلسة)
!pip install gradio

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import gradio as gr
from PIL import Image

# 2. تحميل النموذج الذي رفعته يدوياً من جهازك
# ملاحظة: تأكد أن الاسم مطابق للملف الذي رفعته (fer2013_cnn_model.h5)
try:
    model = load_model('fer2013_cnn_model.h5')
    print("✅ تم تحميل النموذج بنجاح!")
except:
    print("❌ خطأ: لم يتم العثور على ملف النموذج. تأكد من رفعه في قائمة الملفات يميناً.")

# 3. تعريف الأسماء البرمجية للمشاعر
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

def predict_emotion(input_img):
    if input_img is None:
        return "الرجاء رفع صورة"

    # معالجة الصورة: تحويل لرمادي، تغيير الحجم لـ 48x48، ثم تطبيع القيم
    img = Image.fromarray(input_img.astype('uint8'), 'RGB')
    img = img.convert('L')
    img = img.resize((48, 48))

    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    # التنبؤ
    prediction = model.predict(img_array)[0]

    # إرجاع النتائج كقاموس لعرضها في الواجهة
    return {class_labels[i]: float(prediction[i]) for i in range(len(class_labels))}

# 4. إعداد واجهة المستخدم
interface = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Image(),
    outputs=gr.Label(num_top_classes=3, label="التوقعات"),
    title="نظام التعرف على مشاعر الوجه 🧠",
    description="ارفع صورة وجهك هنا وسيقوم الذكاء الاصطناعي بتحليل مشاعرك بناءً على النموذج المرفوع."
)

# 5. انطلاق الواجهة
interface.launch(share=True)

✅ تم تحميل النموذج بنجاح!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8131183e754c17ebfb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
